In [1]:
!pip install xdggs
!pip install healpy
!pip install -U zarr==2.18.4
!pip install numcodecs==0.13.1

In [2]:
import warnings
import healpy as hp
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr
import xdggs
import fsspec

warnings.filterwarnings("ignore")
xr.set_options(display_max_rows=27)

In [3]:
fs = fsspec.filesystem("http")
dtotal=xr.open_zarr("/home/jovyan/Akathon-Geilo/odysea_10_nest.zarr").pipe(xdggs.decode)
#dtotal=xr.open_zarr("https://data-fair2adapt.ifremer.fr/odysea/odysea.zarr").pipe(xdggs.decode)
dtotal

<xarray.Dataset> Size: 52GB
Dimensions:               (cells: 12582912, orbit: 20)
Coordinates:
  * cell_ids              (cells) int64 101MB dask.array<chunksize=(4194304,), meta=np.ndarray>
  * orbit                 (orbit) int64 160B 161 162 163 164 ... 177 178 179 180
Dimensions without coordinates: cells
Data variables:
    azimuth_aft           (cells, orbit) float64 2GB dask.array<chunksize=(4194304, 1), meta=np.ndarray>
    azimuth_fore          (cells, orbit) float64 2GB dask.array<chunksize=(4194304, 1), meta=np.ndarray>
    encoder_aft           (cells, orbit) float64 2GB dask.array<chunksize=(4194304, 1), meta=np.ndarray>
    encoder_fore          (cells, orbit) float64 2GB dask.array<chunksize=(4194304, 1), meta=np.ndarray>
    hit                   (cells, orbit) float64 2GB dask.array<chunksize=(4194304, 1), meta=np.ndarray>
    lat                   (cells, orbit) float64 2GB dask.array<chunksize=(4194304, 1), meta=np.ndarray>
    lon                   (cells, orbit) float64 2GB dask.array<chunksize=(4194304, 1), meta=np.ndarray>
    radial_angle_aft      (cells, orbit) float64 2GB dask.array<chunksize=(4194304, 1), meta=np.ndarray>
    radial_angle_fore     (cells, orbit) float64 2GB dask.array<chunksize=(4194304, 1), meta=np.ndarray>
    sample_time           (cells, orbit) float64 2GB dask.array<chunksize=(4194304, 1), meta=np.ndarray>
    u_model               (cells, orbit) float64 2GB dask.array<chunksize=(4194304, 1), meta=np.ndarray>
    ur_ac                 (cells, orbit) float64 2GB dask.array<chunksize=(4194304, 1), meta=np.ndarray>
    ur_aft                (cells, orbit) float64 2GB dask.array<chunksize=(4194304, 1), meta=np.ndarray>
    ur_al                 (cells, orbit) float64 2GB dask.array<chunksize=(4194304, 1), meta=np.ndarray>
    ur_eastward           (cells, orbit) float64 2GB dask.array<chunksize=(4194304, 1), meta=np.ndarray>
    ur_fore               (cells, orbit) float64 2GB dask.array<chunksize=(4194304, 1), meta=np.ndarray>
    ur_nonoise_ac         (cells, orbit) float64 2GB dask.array<chunksize=(4194304, 1), meta=np.ndarray>
    ur_nonoise_aft        (cells, orbit) float64 2GB dask.array<chunksize=(4194304, 1), meta=np.ndarray>
    ur_nonoise_al         (cells, orbit) float64 2GB dask.array<chunksize=(4194304, 1), meta=np.ndarray>
    ur_nonoise_eastward   (cells, orbit) float64 2GB dask.array<chunksize=(4194304, 1), meta=np.ndarray>
    ur_nonoise_fore       (cells, orbit) float64 2GB dask.array<chunksize=(4194304, 1), meta=np.ndarray>
    ur_nonoise_northward  (cells, orbit) float64 2GB dask.array<chunksize=(4194304, 1), meta=np.ndarray>
    ur_northward          (cells, orbit) float64 2GB dask.array<chunksize=(4194304, 1), meta=np.ndarray>
    v_model               (cells, orbit) float64 2GB dask.array<chunksize=(4194304, 1), meta=np.ndarray>
    wind_dir              (cells, orbit) float64 2GB dask.array<chunksize=(4194304, 1), meta=np.ndarray>
    wind_speed            (cells, orbit) float64 2GB dask.array<chunksize=(4194304, 1), meta=np.ndarray>
Indexes:
    cell_ids  HealpixIndex(nside=10, indexing_scheme=nested)

In [4]:
dtotal.u_model.isel(orbit=0).where(lambda x : x!=hp.UNSEEN).compute().dggs.explore(alpha=0.8)
#dtotal.u_model.where(lambda x : x!=hp.UNSEEN).mean("orbit").compute().dggs.explore(cmap='jet',alpha=0.8)

Map(layers=[SolidPolygonLayer(filled=True, get_fill_color=<pyarrow.lib.FixedSizeListArray object at 0x72151e11…

In [5]:

lat = np.arange(50, 70, 0.03)
lon = np.arange(4, 15, 0.03)
full_lat = np.repeat(lat, len(lon))
full_lon = np.tile(lon, len(lat))
ds=np.sqrt(dtotal.u_model.where(lambda x : x!=hp.UNSEEN)**2+dtotal.v_model.where(lambda x : x!=hp.UNSEEN)**2).dggs.sel_latlon(longitude=full_lon, latitude=full_lat).mean(dim="orbit").compute()#.dggs.explore(alpha=0.8)
ds.dggs.explore(alpha=0.8,cmap='coolwarm')
#dtotal.u_model.where(lambda x : x!=hp.UNSEEN).mean("orbit").compute().dggs.explore(cmap='jet',alpha=0.8)

Map(layers=[SolidPolygonLayer(filled=True, get_fill_color=<pyarrow.lib.FixedSizeListArray object at 0x72151e11…